In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE164883"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE164883"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE164883.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE164883.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE164883.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic heterogeneity of Coeliac Disease biopsies from the duodenum"
!Series_summary	"Here, we present a high-resolution analysis of the transcriptomes extracted from duodenal probes of 25 children and adolescents with active CD and 21 children without CD but with diverse intestinal afflictions as controls. We found that the transcriptomes of CD patients divide into three subgroups, a mixed group resembling part of control cases and a CD-low and CD-high groups referring to lower and higher levels of CD severity"
!Series_summary	"Despite generally increased inflammation, considerable variation in inflammation-level between subgroups was observed, which was further de-chiffred into immune cell types using immune cell de-convolution"
!Series_overall_design	"one biopsy (15–20 mg) was taken from descending duodenum of each patient. Fresh tissue samples were snap frozen and stored in liquid nitrogen until preparation. Frozen biopsies were disrupt

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptome data from duodenal biopsies
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From Sample Characteristics Dictionary:
# Key 0 has disease status: Control or Celiac disease (this is the trait)
# Key 2 has age information
# There's no gender information available
trait_row = 0
age_row = 2
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion
def convert_trait(val):
    """Convert trait value to binary: 1 for Celiac disease, 0 for Control."""
    if val is None:
        return None
    if ':' in val:
        val = val.split(':', 1)[1].strip()
    if val.lower() == 'celiac disease':
        return 1
    elif val.lower() == 'control':
        return 0
    return None

def convert_age(val):
    """Convert age value to continuous numeric value."""
    if val is None:
        return None
    if ':' in val:
        val = val.split(':', 1)[1].strip()
    try:
        return float(val)
    except (ValueError, TypeError):
        return None

def convert_gender(val):
    """Convert gender to binary: 0 for female, 1 for male."""
    # Not used as gender data is not available
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
# Conduct initial filtering and save information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # clinical_data is presumed to be available from previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to the specified output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM5022362': [0.0, 1.0], 'GSM5022363': [1.0, 3.0], 'GSM5022364': [1.0, 3.0], 'GSM5022365': [1.0, 4.0], 'GSM5022366': [1.0, 7.0], 'GSM5022367': [0.0, 6.0], 'GSM5022368': [1.0, 9.0], 'GSM5022369': [0.0, 10.0], 'GSM5022370': [1.0, 10.0], 'GSM5022371': [0.0, 11.0], 'GSM5022372': [1.0, 11.0], 'GSM5022373': [1.0, 12.0], 'GSM5022374': [0.0, 12.0], 'GSM5022375': [0.0, 12.0], 'GSM5022376': [0.0, 12.0], 'GSM5022377': [0.0, 13.0], 'GSM5022378': [1.0, 13.0], 'GSM5022379': [1.0, 13.0], 'GSM5022380': [0.0, 14.0], 'GSM5022381': [0.0, 15.0], 'GSM5022382': [1.0, 16.0], 'GSM5022383': [1.0, 16.0], 'GSM5022384': [0.0, 17.0], 'GSM5022385': [1.0, 17.0], 'GSM5022386': [0.0, 17.0], 'GSM5022387': [1.0, 3.0], 'GSM5022388': [0.0, 2.0], 'GSM5022389': [0.0, 2.0], 'GSM5022390': [1.0, 3.0], 'GSM5022391': [1.0, 4.0], 'GSM5022392': [1.0, 4.0], 'GSM5022393': [1.0, 5.0], 'GSM5022394': [1.0, 4.0], 'GSM5022395': [0.0, 7.0], 'GSM5022396': [1.0, 8.0], 'GSM5022397': [1.0, 12.0], 'GSM5

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Celiac_Disease/GSE164883/GSE164883_series_matrix.txt.gz


Gene data shape: (47307, 48)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# These identifiers (ILMN_*) are Illumina microarray probe IDs, not human gene symbols
# They need to be mapped to standard gene symbols for biological interpretation

# Based on my biomedical knowledge, ILMN_* identifiers are Illumina BeadArray probe IDs
# and require mapping to human gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in gene annotation - 'ID' for probe identifiers and 'Symbol' for gene symbols
probe_id_col = 'ID'
gene_symbol_col = 'Symbol'

print(f"Using {probe_id_col} for probe identifiers and {gene_symbol_col} for gene symbols")

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Sample of gene mapping data:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene-level expression by applying gene mapping
# This handles the many-to-many relationship between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(list(gene_data.index[:10]))

# Optionally save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Using ID for probe identifiers and Symbol for gene symbols
Gene mapping shape: (44837, 2)
Sample of gene mapping data:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB


Gene expression data shape after mapping: (21460, 48)
First few gene symbols after mapping:
['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT']


Gene expression data saved to ../../output/preprocess/Celiac_Disease/gene_data/GSE164883.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Re-load clinical data from matrix file to ensure we have the correct data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Re-extract clinical features with the properly loaded clinical data
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function
try:
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    print("Linked data first few columns:")
    print(linked_data.columns[:10])
    
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Conduct quality check and save the cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=unbiased_linked_data,
        note="Dataset contains gene expression from duodenal biopsies of Celiac Disease patients, CVID patients, and healthy controls"
    )
    
    # 6. If the linked data is usable, save it as a CSV file
    if is_usable:
        print(f"Data is usable. Saving to {out_data_file}")
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
    else:
        print("Data is not usable. Not saving linked data file.")
        
except Exception as e:
    print(f"Error in data linking or processing: {e}")
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True, 
        df=pd.DataFrame(),
        note=f"Error processing data: {e}"
    )
    print("Data is not usable due to processing error.")

Clinical data shape: (2, 48)
Clinical data preview:
{'GSM5022362': [0.0, 1.0], 'GSM5022363': [1.0, 3.0], 'GSM5022364': [1.0, 3.0], 'GSM5022365': [1.0, 4.0], 'GSM5022366': [1.0, 7.0], 'GSM5022367': [0.0, 6.0], 'GSM5022368': [1.0, 9.0], 'GSM5022369': [0.0, 10.0], 'GSM5022370': [1.0, 10.0], 'GSM5022371': [0.0, 11.0], 'GSM5022372': [1.0, 11.0], 'GSM5022373': [1.0, 12.0], 'GSM5022374': [0.0, 12.0], 'GSM5022375': [0.0, 12.0], 'GSM5022376': [0.0, 12.0], 'GSM5022377': [0.0, 13.0], 'GSM5022378': [1.0, 13.0], 'GSM5022379': [1.0, 13.0], 'GSM5022380': [0.0, 14.0], 'GSM5022381': [0.0, 15.0], 'GSM5022382': [1.0, 16.0], 'GSM5022383': [1.0, 16.0], 'GSM5022384': [0.0, 17.0], 'GSM5022385': [1.0, 17.0], 'GSM5022386': [0.0, 17.0], 'GSM5022387': [1.0, 3.0], 'GSM5022388': [0.0, 2.0], 'GSM5022389': [0.0, 2.0], 'GSM5022390': [1.0, 3.0], 'GSM5022391': [1.0, 4.0], 'GSM5022392': [1.0, 4.0], 'GSM5022393': [1.0, 5.0], 'GSM5022394': [1.0, 4.0], 'GSM5022395': [0.0, 7.0], 'GSM5022396': [1.0, 8.0], 'GSM5022397': [1.0,

Linked data shape after handling missing values: (48, 21462)
For the feature 'Celiac_Disease', the least common label is '0.0' with 22 occurrences. This represents 45.83% of the dataset.
The distribution of the feature 'Celiac_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 4.75
  50% (Median): 12.0
  75%: 14.0
Min: 1.0
Max: 17.0
The distribution of the feature 'Age' in this dataset is fine.

Data is usable. Saving to ../../output/preprocess/Celiac_Disease/GSE164883.csv
